In [1]:
from pycaret.datasets import get_data
dataset = get_data('credit')

,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_1,PAY_2,PAY_3,PAY_4,PAY_5,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default
0,20000,2,2,1,24,2,2,-1,-1,-2,...,0.0,0.0,0.0,0.0,689.0,0.0,0.0,0.0,0.0,1
1,90000,2,2,2,34,0,0,0,0,0,...,14331.0,14948.0,15549.0,1518.0,1500.0,1000.0,1000.0,1000.0,5000.0,0
2,50000,2,2,1,37,0,0,0,0,0,...,28314.0,28959.0,29547.0,2000.0,2019.0,1200.0,1100.0,1069.0,1000.0,0
3,50000,1,2,1,57,-1,0,-1,0,0,...,20940.0,19146.0,19131.0,2000.0,36681.0,10000.0,9000.0,689.0,679.0,0
4,50000,1,1,2,37,0,0,0,0,0,...,19394.0,19619.0,20024.0,2500.0,1815.0,657.0,1000.0,1000.0,800.0,0


In [2]:
dataset.shape

(24000, 24)

In [3]:
data = dataset.sample(frac=0.95)
data_unseen = dataset.drop(data.index)
data.reset_index(inplace=True, drop=True)
data_unseen.reset_index(inplace=True, drop=True)
print('Data for Modeling: ' + str(data.shape))
print('Unseen Data For Predictions: ' + str(data_unseen.shape))

Data for Modeling: (22800, 24)
Unseen Data For Predictions: (1200, 24)


In [4]:
from pycaret.classification import *

In [5]:
exp_clf101 = setup(data = data, target = 'default', session_id=123, use_gpu=True)

,Description,Value
0,session_id,123
1,Target,default
2,Target Type,Binary
3,Label Encoded,None
4,Original Data,"(22800, 24)"
5,Missing Values,0
6,Numeric Features,14
7,Categorical Features,9
8,Ordinal Features,0
9,High Cardinality Features,0


In [6]:
best_model = compare_models()

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
ridge,Ridge Classifier,0.8225,0.0000,0.3612,0.6799,0.4715,0.3766,0.4040,0.0360
lda,Linear Discriminant Analysis,0.8217,0.7664,0.3778,0.6648,0.4815,0.3835,0.4061,0.2470
gbc,Gradient Boosting Classifier,0.8212,0.7814,0.3652,0.6706,0.4725,0.3760,0.4014,5.9470
catboost,CatBoost Classifier,0.8199,0.7779,0.3744,0.6573,0.4767,0.3777,0.3997,7.9480
lightgbm,Light Gradient Boosting Machine,0.8190,0.7750,0.3690,0.6562,0.4719,0.3729,0.3957,2.3090
ada,Ada Boost Classifier,0.8162,0.7714,0.3249,0.6671,0.4363,0.3419,0.3738,1.4040
xgboost,Extreme Gradient Boosting,0.8086,0.7597,0.3532,0.6098,0.4472,0.3412,0.3599,1.0210
lr,Logistic Regression,0.7808,0.6421,0.0000,0.0000,0.0000,0.0000,0.0000,0.5030
dummy,Dummy Classifier,0.7808,0.5000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0140
svm,SVM - Linear Kernel,0.7527,0.0000,0.0929,0.2629,0.0806,0.0281,0.0379,0.5110


In [7]:
print(best_model)

RidgeClassifier(alpha=1.0, class_weight=None, copy_X=True, fit_intercept=True,
                max_iter=None, normalize=False, random_state=123, solver='auto',
                tol=0.001)


In [8]:
models()

,Name,Reference,Turbo
ID,,,
lr,Logistic Regression,sklearn.linear_model._logistic.LogisticRegression,True
knn,K Neighbors Classifier,sklearn.neighbors._classification.KNeighborsCl...,True
nb,Naive Bayes,sklearn.naive_bayes.GaussianNB,True
dt,Decision Tree Classifier,sklearn.tree._classes.DecisionTreeClassifier,True
svm,SVM - Linear Kernel,sklearn.linear_model._stochastic_gradient.SGDC...,True
rbfsvm,SVM - Radial Kernel,sklearn.svm._classes.SVC,False
gpc,Gaussian Process Classifier,sklearn.gaussian_process._gpc.GaussianProcessC...,False
mlp,MLP Classifier,sklearn.neural_network._multilayer_perceptron....,False
ridge,Ridge Classifier,sklearn.linear_model._ridge.RidgeClassifier,True


In [9]:
unseen_predictions = predict_model(best_model, data=data_unseen)
unseen_predictions.head()

,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_1,PAY_2,PAY_3,PAY_4,PAY_5,...,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default,Label
0,20000,2,2,1,24,2,2,-1,-1,-2,...,0.0,0.0,0.0,689.0,0.0,0.0,0.0,0.0,1,1
1,250000,1,1,2,29,0,0,0,0,0,...,56875.0,55512.0,3000.0,3000.0,3000.0,3000.0,3000.0,3000.0,0,0
2,50000,1,3,2,25,-1,0,0,0,0,...,9590.0,10030.0,1759.0,1779.0,320.0,500.0,1000.0,1000.0,0,0
3,350000,1,1,2,52,-1,-1,-1,-1,-1,...,1150.0,5263.0,2272.0,722.0,867.0,1150.0,5263.0,5011.0,0,0
4,210000,2,2,1,33,0,0,0,0,0,...,4404.0,5708.0,1500.0,1500.0,1000.0,500.0,2000.0,546.0,0,0


In [10]:
from pycaret.utils import check_metric
check_metric(unseen_predictions['default'], unseen_predictions['Label'], metric = 'Accuracy')

0.8292

In [11]:
save_model(best_model,'Pycaret_Ridge_Classifier')

Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=None,
          steps=[('dtypes',
                  DataTypes_Auto_infer(categorical_features=[],
                                       display_types=True, features_todrop=[],
                                       id_columns=[],
                                       ml_usecase='classification',
                                       numerical_features=[], target='default',
                                       time_features=[])),
                 ('imputer',
                  Simple_Imputer(categorical_strategy='not_available',
                                 fill_value_categorical=None,
                                 fill_value_numerical=None,
                                 numeric_stra...
                 ('fix_perfect', Remove_100(target='default')),
                 ('clean_names', Clean_Colum_Names()),
                 ('feature_select', 'passthrough'), ('fix_multi', 'passthrough'),
                 ('dfs', 'passthrough'), ('pca', 'passthrough'),
        